 <font color=black size=5>**1.Piptides data clean<br>
    2. peptides maxLFQ<br>
 3. protein missing value impute<br>
 4. protein batch correction<br>
 5. QC**<br>

# 1. Piptides data clean

In [2]:
import pandas as pd
import openpyxl
import xlsxwriter
import numpy as np
import os
import re
import fancyimpute

In [3]:
report_data = "/home/zhl/projects/batch_effect_correction/new_results/"
os.chdir(report_data)
#group path
group_data = "/home/zhl/projects/batch_effect_correction/new_results/group_new.xls"

### 1.1  EG.PrecursorId 

In [4]:
data = pd.read_table("20180509_104033_Multiple site study_ReportPeptides.xls",low_memory=False,na_values="Filtered")
data['EG.PrecursorId']=data['EG.PrecursorId'].str.replace("_.\d$",'',regex=True)
df = data.drop_duplicates(subset=["EG.PrecursorId","PG.FASTAName"])
df = df.drop(df.columns[3],axis=1)
df.reset_index(drop=True,inplace = True)

### 1.2  clean samples' name

In [5]:
df_name = []
for col in df.columns:
    if "]" in col:
        col = col.split("] ",1)[1].replace("-","_").replace("Lab_",'')
    else:
        col = col.split("] ",1)[0]
    df_name.append(col)
df.columns = df_name

In [8]:
df[df.columns[df.columns.str.contains('EG.Qvalue')]].max(skipna=True)#Qvalue <= 1%

ASround1_Day_1_Sample_A_Rep_1.raw.EG.Qvalue    0.009999
ASround1_Day_1_Sample_A_Rep_2.raw.EG.Qvalue    0.009997
ASround1_Day_1_Sample_A_Rep_3.raw.EG.Qvalue    0.009999
ASround1_Day_1_Sample_B_Rep_1.raw.EG.Qvalue    0.009996
ASround1_Day_1_Sample_B_Rep_2.raw.EG.Qvalue    0.009999
                                                 ...   
VUMC_Day_7_Sample_A_Rep_2.raw.EG.Qvalue        0.009989
VUMC_Day_7_Sample_A_Rep_3.raw.EG.Qvalue        0.009997
VUMC_Day_7_Sample_B_Rep_1.raw.EG.Qvalue        0.010000
VUMC_Day_7_Sample_B_Rep_2.raw.EG.Qvalue        0.010000
VUMC_Day_7_Sample_B_Rep_3.raw.EG.Qvalue        0.009997
Length: 305, dtype: float64

### 1.3   protein & peptides data  filtered sample species

In [28]:
## 筛选 物种；sample种类
'''
species = df["PG.FASTAName"].unique()
print(species)

df_ = df[df["PG.FASTAName"]==species[0]]

#drop sample_B
df_protein = df_[df_.columns[~df_.columns.str.contains('EG.Qvalue|PEP.Quantity|Sample_B')]]
df_protein = df_protein.drop(df_protein.columns[[1,3]],axis=1)
'''

'\nspecies = df["PG.FASTAName"].unique()\nprint(species)\n\ndf_ = df[df["PG.FASTAName"]==species[0]]\n\n#drop sample_B\ndf_protein = df_[df_.columns[~df_.columns.str.contains(\'EG.Qvalue|PEP.Quantity|Sample_B\')]]\ndf_protein = df_protein.drop(df_protein.columns[[1,3]],axis=1)\n'

### 1.4.1 Protein

In [9]:
#delete Qvalue, peptide
df_protein = df[df.columns[~df.columns.str.contains('EG.Qvalue|PEP.Quantity')]]
df_protein = df_protein.drop_duplicates(subset=["PG.FASTAName","PG.ProteinAccessions"])
df_protein = df_protein.drop("EG.PrecursorId",axis=1)

### 1.4.2 Peptides 

In [10]:
df_pep = df[df.columns[~df.columns.str.contains('EG.Qvalue|PG.Quantity')]]

df_pep_name = []
for col in df_pep.columns:
    col = col.split(".raw.PEP.Quantity",1)[0]
    col = re.sub(r'Sample_', '', col)
    df_pep_name.append(col)
df_pep.columns = df_pep_name

df_pep.index = df_pep["EG.PrecursorId"]

# 2. Peptides maxLFQ

In [11]:
df_pep_num = df_pep.iloc[:,3:]
df_pep_num[df_pep_num <= 100] = np.nan
df_pep_num.dropna(axis=1,thresh=100,inplace = True)
df_pep_num.dropna(axis=0,thresh=1,inplace = True)

In [12]:
raw_pep = pd.merge(df_pep.loc[:,["EG.PrecursorId","PG.FASTAName","PG.ProteinAccessions"]],df_pep_num,
                         how="right",left_index = True,right_index = True)

In [13]:
#Save long format pipetides_raw.csv
raw_long = raw_pep.melt(id_vars = raw_pep.columns[0:3],var_name="R.FileName",value_name="PEP.Quantity")
raw_long.dropna().round(3).to_csv("/home/zhl/projects/batch_effect_correction/new_results/peptides_raw_long.csv")

**<font color=green size=4>raw peptide data MaxLFQ**

In [14]:
os.system('Rscript /home/zhl/projects/batch_effect_correction/new_results/R/1.maxLFQ_proteingroup.R {0} {1}'.format("/home/zhl/projects/batch_effect_correction/new_results/peptides_raw_long.csv",
                                                                                                  "/home/zhl/projects/batch_effect_correction/new_results/Protein_raw_maxLFQ.csv"))



载入程辑包：‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


载入程辑包：‘data.table’

The following objects are masked from ‘package:dplyr’:

    between, first, last

Concatenating secondary ids...

Removing low intensities...



nrow = 15946548, # proteins = 9164, # samples = 305
23 thread(s) available...
0%
5%
10%
15%
21%
26%
31%
36%
42%
47%
52%
57%
62%
67%
72%
78%
83%
88%
93%
98%
Completed.


0

# 2. missing value process

**peptide** missing value<60% knn impute missing value

In [17]:
raw_pep.index = raw_pep["EG.PrecursorId"]
raw_pep_num = raw_pep.iloc[:,3:]
raw_pep_num[raw_pep_num <= 100] = np.nan
raw_pep_num.dropna(axis=0,thresh=raw_pep_num.shape[1]*0.4,inplace = True)
raw_pep_num = raw_pep_num.T
raw_pep_impute = fancyimpute.KNN(k=15).fit_transform(raw_pep_num.values) #knn按列进行填充
raw_pep_impute = pd.DataFrame(raw_pep_impute, columns=list(raw_pep_num.columns), index=raw_pep_num.index).T

pep_impute = pd.merge(raw_pep.loc[:,["EG.PrecursorId","PG.FASTAName","PG.ProteinAccessions"]],raw_pep_impute,
                         how="right",left_index = True,right_index = True)
pep_impute.round(3).to_csv("/home/zhl/projects/batch_effect_correction/new_results/peptides_raw_matrix_impute0.4.csv",index=False)

Imputing row 1/305 with 9850 missing, elapsed time: 26.082
Imputing row 101/305 with 8814 missing, elapsed time: 29.891
Imputing row 201/305 with 22021 missing, elapsed time: 34.549
Imputing row 301/305 with 14422 missing, elapsed time: 38.335


## Protein

In [18]:
df_protein = pd.read_csv("/home/zhl/projects/batch_effect_correction/new_results/Protein_raw_maxLFQ.csv",
                        low_memory=False) #index: Protein

<font color=red size=3>missing value < 50%

In [19]:
df_protein.index = df_protein["Protein"]
df_protein_num = df_protein.iloc[:,2:]
df_protein_num[df_protein_num <= 100] = np.nan
df_protein_num.dropna(axis=0,thresh=df_protein_num.shape[1]*0.5,inplace = True) #or 0.4
df_protein_num = df_protein_num.T
#df_protein_num.fillna(1,inplace=True)

<font color=red size=3>knn impute missing value

In [20]:
df_protein_impute = fancyimpute.KNN(k=15).fit_transform(df_protein_num.values) #knn按列进行填充
df_protein_impute = pd.DataFrame(df_protein_impute, columns=list(df_protein_num.columns), index=df_protein_num.index).T

Imputing row 1/305 with 904 missing, elapsed time: 2.388
Imputing row 101/305 with 825 missing, elapsed time: 2.723
Imputing row 201/305 with 2531 missing, elapsed time: 3.137
Imputing row 301/305 with 811 missing, elapsed time: 3.438


In [21]:
dsr = pd.merge(df_protein.loc[:,["Protein","PG.FASTAName"]],df_protein_impute,
                         how="right",left_index = True,right_index = True)

In [22]:
#Save
#col = [0]+list(range(2,dsr.shape[1]))
dsr.round(3).to_csv("/home/zhl/projects/batch_effect_correction/new_results/Protein_raw_maxLFQ_impute0.4.csv",index=False)
dsr.T.iloc[2:,:].round(3).to_csv("/home/zhl/projects/batch_effect_correction/new_results/Protein_raw_maxLFQ_impute0.4_T.csv")
del [data,df_pep,df_protein,dsr]

# 3. batch effect correct

In [4]:
# R
#"/home/boot/Script/test/batch_pipeline/R"
# 2.data_preprocess.R (normalization; group information) save ds.RData
os.system('Rscript /home/zhl/projects/batch_effect_correction/new_results/R/2.data_preprocess.R {0} {1} {2}'.format("/home/zhl/projects/batch_effect_correction/new_results/Protein_raw_maxLFQ_impute0.4_T.csv",
                                                                                                                  group_data,
                                                                                                                  "/home/zhl/projects/batch_effect_correction/new_results/"))



载入程辑包：‘dplyr’

The following objects are masked from ‘package:data.table’:

    between, first, last

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



0

In [8]:
# 3.batch_Correction.R
os.system('Rscript /home/zhl/projects/batch_effect_correction/new_results/R/3.batch_Correction_method.R {0} {1} {2}'.format(
                        "/home/zhl/projects/batch_effect_correction/new_results/ds.RData",
                        "/home/zhl/projects/batch_effect_correction/new_results/Protein_raw_maxLFQ_impute0.4.csv",
                        "/home/zhl/projects/batch_effect_correction/new_results/protein_after_batch/"))

[1] "raw finish!!"


载入需要的程辑包：mgcv
载入需要的程辑包：nlme
This is mgcv 1.8-40. For overview type 'help("mgcv-package")'.
载入需要的程辑包：genefilter
载入需要的程辑包：BiocParallel
Found12batches
Adjusting for0covariate(s) or covariate level(s)
Standardizing Data across genes
Fitting L/S model and finding priors
Finding parametric adjustments
Adjusting the Data



[1] "Combat finish!!"


载入需要的程辑包：Biobase
载入需要的程辑包：BiocGenerics

载入程辑包：‘BiocGenerics’

The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs

The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, Map, mapply, match, mget,
    order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rownames, sapply, setdiff, sort, table, tapply,
    union, unique, unsplit, which.max, which.min

Welcome to Bioconductor

    Vignettes contain introductory material; view with
    'browseVignettes()'. To cite Bioconductor, see
    'citation("Biobase")', and for packages 'citation("pkgname")'.

载入需要的程辑包：EDASeq
载入需要的程辑包：ShortRead
载入需要的程辑包：Biostrings
载入需要的程辑包：S4Vectors
载入需要的程辑包：stats4

载入程辑包：‘S4Vectors’

The following objects are masked from ‘package:data.table’:

    first, second

The following o

[1] "RUVs finish!!"
[1] "RUVg finish!!"
[1] "RUVIII finish!!"



载入程辑包：‘dplyr’

The following object is masked from ‘package:ShortRead’:

    id

The following objects are masked from ‘package:GenomicAlignments’:

    first, last

The following object is masked from ‘package:matrixStats’:

    count

The following objects are masked from ‘package:GenomicRanges’:

    intersect, setdiff, union

The following objects are masked from ‘package:Biostrings’:

    collapse, intersect, setdiff, setequal, union

The following object is masked from ‘package:GenomeInfoDb’:

    intersect

The following object is masked from ‘package:XVector’:

    slice

The following objects are masked from ‘package:IRanges’:

    collapse, desc, intersect, setdiff, slice, union

The following objects are masked from ‘package:S4Vectors’:

    first, intersect, rename, setdiff, setequal, union

The following object is masked from ‘package:Biobase’:

    combine

The following objects are masked from ‘package:BiocGenerics’:

    combine, intersect, setdiff, union

The followin

[1] "Median-Norm finish!!"
[1] "QC-Mnorm finish!!"
[1] "Limma finish!!"


0

# 5. QC (PCA,boxplot)

## 5.1 Protein

### 5.1.1  all samples and all species

In [45]:
path_QC = "/home/zhl/projects/batch_effect_correction/new_results/QC/"
if not os.path.exists(os.path.join(path_QC,"PCA")):
    os.makedirs(os.path.join(path_QC,"PCA"))
    os.makedirs(os.path.join(path_QC,"boxplot"))
    os.makedirs(os.path.join(path_QC,"pointplot"))
    os.makedirs(os.path.join(path_QC,"RSD"))
    os.makedirs(os.path.join(path_QC,"variancePartition"))
    os.makedirs(os.path.join(path_QC,"Hcluster"))

In [10]:
path_pro = "/home/zhl/projects/batch_effect_correction/new_results/"
files= os.listdir(os.path.join(path_pro,"protein_after_batch"))
for i in files:
    if os.path.splitext(i)[1] == '.csv':
        file = os.path.join(path_pro,"protein_after_batch/",i)
        print(file)
        os.system('Rscript /home/zhl/projects/batch_effect_correction/new_results/R/4.QC.R {0} {1} {2}'.format(file,
                                                                                              group_data,               
                                                                                      os.path.join(path_pro,"QC/")))
        print("=====================")

/home/zhl/projects/batch_effect_correction/new_results/protein_after_batch/protein_after_batch_RUVIII.csv


Warning message:
Removed 13019 rows containing non-finite values (stat_boxplot). 
Warning message:
Removed 13019 rows containing non-finite values (stat_boxplot). 


/home/zhl/projects/batch_effect_correction/new_results/protein_after_batch/protein_after_batch_RUVg.csv


Warning message:
Removed 9539 rows containing non-finite values (stat_boxplot). 
Warning message:
Removed 9539 rows containing non-finite values (stat_boxplot). 


/home/zhl/projects/batch_effect_correction/new_results/protein_after_batch/protein_after_batch_Combat.csv


...

Warning message:
Removed 18501 rows containing non-finite values (stat_boxplot). 
Warning message:
Removed 18501 rows containing non-finite values (stat_boxplot). 


/home/zhl/projects/batch_effect_correction/new_results/protein_after_batch/protein_after_batch_Limma.csv


Warning message:
Removed 10421 rows containing non-finite values (stat_boxplot). 
Warning message:
Removed 10421 rows containing non-finite values (stat_boxplot). 


/home/zhl/projects/batch_effect_correction/new_results/protein_after_batch/protein_after_batch_Median-Norm.csv


Warning message:
Removed 10865 rows containing non-finite values (stat_boxplot). 
Warning message:
Removed 10865 rows containing non-finite values (stat_boxplot). 


### 5.1.2  sample (A;B) and each specie

In [45]:
path_pro = "/home/boot/Script/test/batch_pipeline/NC_batch_pipeline2/Peptide/"
files= os.listdir("/home/boot/Script/test/batch_pipeline/NC_batch_pipeline2/Peptide/protein_after_batch/")
samples = ["Sample_A","Sample_B"]
for i in files:
    if os.path.splitext(i)[1] == '.csv':
        file = os.path.join("/home/boot/Script/test/batch_pipeline/NC_batch_pipeline2/Peptide/protein_after_batch/",i)
        protein = pd.read_csv(file)
        species = protein["PG.FASTAName"].unique()
        for s in species:
            protein1 = protein[protein["PG.FASTAName"]==s]
            if s == "Saccharomyces cerevisiae_201602":
                s = "Yeast"
            for sample in samples:
                if sample == "Sample_B":
                    sample = "sample_B"
                    #drop sample_A
                    protein_s = protein1[protein1.columns[~protein1.columns.str.contains('Sample_A')]]
                else:
                    sample = "sample_A"
                    #drop sample_B
                    protein_s = protein1[protein1.columns[~protein1.columns.str.contains('Sample_B')]]
                filename = os.path.join(path_pro,s,sample,"Protein/",i)
                print(filename)
                protein_s.to_csv(filename,index=False)
                os.system('Rscript /home/boot/Script/test/batch_pipeline/NC_batch_pipeline2/4.QC.R {0} {1}'.format(filename,
                                                                                          os.path.join(path_pro,s,sample,"Protein/")))
                print("=====================")

/home/boot/Script/test/batch_pipeline/NC_batch_pipeline2/Peptide/swissprot_homo_201604/sample_A/Protein/protein_after_batch_QC-Mnorm.csv


FileNotFoundError: [Errno 2] No such file or directory: '/home/boot/Script/test/batch_pipeline/NC_batch_pipeline2/Peptide/swissprot_homo_201604/sample_A/Protein/protein_after_batch_QC-Mnorm.csv'

### 5.1.3  Pearson correlation

In [68]:
os.system('Rscript /home/zhl/projects/batch_effect_correction/new_results/R/9.pearson_method.R {0}'.format("/home/zhl/projects/batch_effect_correction/new_results/protein_after_batch/"))

Warning message:
Removed 58 rows containing non-finite values (stat_boxplot). 
Warning message:
Removed 58 rows containing non-finite values (stat_boxplot). 
Warning messages:
1: Removed 58 rows containing non-finite values (stat_ydensity). 
2: Removed 58 rows containing non-finite values (stat_boxplot). 
Warning messages:
1: Removed 58 rows containing non-finite values (stat_ydensity). 
2: Removed 58 rows containing non-finite values (stat_boxplot). 


0

## 5.2 Peptide 

### 5.2.1  all samples and all species

In [ ]:
path_QC = "/home/boot/Script/test/batch_pipeline/NC_batch_pipeline2/Peptide/QC/Peptide/"
if not os.path.exists(os.path.join(path_QC,"PCA")):
    os.makedirs(os.path.join(path_QC,"boxplot"))
    os.makedirs(os.path.join(path_QC,"pointplot"))
    os.makedirs(os.path.join(path_QC,"RSD"))
    os.makedirs(os.path.join(path_QC,"variancePartition"))
    os.makedirs(os.path.join(path_QC,"Hcluster"))

In [ ]:
path_pep = "/home/boot/Script/test/batch_pipeline/NC_batch_pipeline2/Peptide/"
file= "/home/boot/Script/test/batch_pipeline/NC_batch_pipeline2/Peptide/peptides_raw_matrix.csv"
os.system('Rscript /home/zhl/projects/batch_effect_correction/new_results/R/4.QC.R {0} {1} {2}'.format(file,group_data,
                                                                                      os.path.join(path_pep,"QC/Peptide")))
print("=====================")

## 6 RSD 

## 6.1 Protein + Peptide **<font color = red>all samples and all species**

### RSD was calculated separately for each lab

In [5]:
import pandas as pd
#import xlrd
MSFile = pd.read_csv('/home/zhl/projects/batch_effect_correction/new_results/protein_after_batch/protein_after_batch_RUVIII.csv',low_memory=False)
group=pd.read_excel("/home/zhl/projects/batch_effect_correction/new_results/group_new.xls",header=None)
group = group.iloc[1:,0:2]
group = group.merge(pd.DataFrame(MSFile.columns,columns=[0]))
group

,0,1
0,FMMU_Day_1_A_rep_1,Lab1
1,FMMU_Day_1_A_rep_2,Lab1
2,FMMU_Day_1_A_rep_3,Lab1
3,FMMU_Day_1_B_rep_1,Lab1
4,FMMU_Day_1_B_rep_2,Lab1
...,...,...
300,MRL_Day_7_A_Rep_2,Lab11
301,MRL_Day_7_A_Rep_3,Lab11
302,MRL_Day_7_B_Rep_1,Lab11
303,MRL_Day_7_B_Rep_2,Lab11


In [73]:
pro_pep = ["Protein/","Peptide/"]
for p in pro_pep:
    if p == "Peptide/":
        path = "/home/zhl/projects/batch_effect_correction/new_results/QC/Peptide/"
        rsd_path = '/home/zhl/projects/batch_effect_correction/new_results/QC/Peptide/RSD/'
        files = ["peptides_raw_matrix_impute0.4.csv"]
    else:
        path = "/home/zhl/projects/batch_effect_correction/new_results/protein_after_batch/"
        rsd_path = '/home/zhl/projects/batch_effect_correction/new_results/QC/RSD/'
        file_dirs = os.listdir(path)
        files = [i for i in file_dirs if os.path.splitext(i)[1] == '.csv']
    for i in files:
        file = path+i
        print(file)
        MSFile = pd.read_csv(file,low_memory=False)
        group2sample = group.groupby(1,as_index=False)
        # 计算相对标准差RSD
        rsd_data = pd.DataFrame()
        for key, value in group2sample:
            Value = list(value[0])
            rsd_data[key] = MSFile[Value].apply(lambda x: np.std(x[Value], ddof=1) / np.mean(x[Value]),axis=1)
        pd.melt(rsd_data.dropna()).to_csv(os.path.join(rsd_path, 'rsd_'+i),index=None)
        print("save file finish !!!")
        os.system('Rscript /home/zhl/projects/batch_effect_correction/new_results/R/5.RSD.R {0} {1}'.format(os.path.join(rsd_path, 'rsd_'+i),rsd_path))

/home/zhl/projects/batch_effect_correction/new_results/protein_after_batch/protein_after_batch_RUVIII.csv
save file finish !!!
/home/zhl/projects/batch_effect_correction/new_results/protein_after_batch/protein_after_batch_RUVg.csv



KeyboardInterrupt



### **RSD (All labs are considered as a group)**

In [7]:
path = "/home/zhl/projects/batch_effect_correction/new_results/protein_after_batch/"
rsd_path = '/home/zhl/projects/batch_effect_correction/new_results/QC/RSD/'
file_dirs = os.listdir(path)
files = [i for i in file_dirs if os.path.splitext(i)[1] == '.csv']
rsd_data = pd.DataFrame()
for i in files:
    print(i)
    file = path+i
    lab = i.split("protein_after_batch_")[1].split(".csv")[0]
    MSFile = pd.read_csv(file,low_memory=False)
    MSFile = MSFile.loc[:,group[0]]
    rsd_data[lab] = MSFile.apply(lambda x: np.std(x,ddof=1) / np.mean(x),axis=1)
pd.melt(rsd_data.dropna()).to_csv(os.path.join(rsd_path, "RSD_method.csv"),index=None)
os.system('Rscript /home/zhl/projects/batch_effect_correction/new_results/R/6.RSD_mod.R {0} {1}'.format(os.path.join(rsd_path, "RSD_method.csv"),
                                                                                                          rsd_path))
print(p,"finish")

protein_after_batch_RUVIII.csv
protein_after_batch_RUVg.csv
protein_after_batch_Combat.csv
protein_after_batch_QC-Mnorm.csv
protein_after_batch_raw.csv
protein_after_batch_RUVs.csv
protein_after_batch_Limma.csv
protein_after_batch_Median-Norm.csv
Protein/ finish


In [8]:
os.system('Rscript /home/zhl/projects/batch_effect_correction/new_results/R/6.RSD_mod.R {0} {1}'.format(os.path.join(rsd_path, "RSD_method.csv"),
                                                                                                          rsd_path))

0

~~ **<font color = red size=5>6.2 Protein + Peptide each sample and each species** ~~

In [ ]:
'''
group=pd.read_excel("/home/boot/Script/test/batch_pipeline/batch_effect_correction/NC/group.xls",header=None)
group = group.iloc[1:,0:2]
path = "/home/boot/Script/test/batch_pipeline/batch_effect_correction/NC/"
species_dirs = ["swissprot_homo_201604","uniprot-Ecoli_K12_201709","Yeast"]
samples = ["sample_A","sample_B"]
pro_pep = ["Protein/","Peptide/"]
for dirs in species_dirs:
    for sample in samples:
        for p in pro_pep:
            path_dir = os.path.join(path,dirs,sample,p)
            files= os.listdir(path_dir)
            print(path_dir)
            for i in files:
                if os.path.splitext(i)[1] == '.csv':
                    rsd_path = os.path.join(path_dir,"RSD/")
                    file = path_dir+i
                    MSFile = pd.read_csv(file,low_memory=False)
                    group_merge = group.merge(pd.DataFrame(MSFile.columns,columns=[0]))
                    group2sample = group_merge.groupby(1,as_index=False)
                    # 计算相对标准差RSD
                    rsd_data = pd.DataFrame()
                    for key, value in group2sample:
                        #print("key:",key,'value:',value)
                        Value = list(value[0])
                        rsd_data[key] = MSFile[Value].apply(lambda x: np.std(x[Value], ddof=1) / np.mean(x[Value]),axis=1)
                    pd.melt(rsd_data.dropna()).to_csv(os.path.join(rsd_path, 'rsd_'+i),index=None)
                    os.system('Rscript /home/zhl/projects/batch_effect_correction/new_results/R、RSD.R {0} {1}'.format(os.path.join(rsd_path, 'rsd_'+i),
                                                                                                     os.path.join(rsd_path, 'plot/')))
'''

~~ **RSD (All labs are considered as a group)** ~~

In [ ]:
'''
group=pd.read_excel("/home/boot/Script/test/batch_pipeline/batch_effect_correction/NC/group.xls",header=None)
group = group.iloc[1:,0:2]
path = "/home/boot/Script/test/batch_pipeline/batch_effect_correction/NC/"
species_dirs = ["swissprot_homo_201604","uniprot-Ecoli_K12_201709","Yeast"]
samples = ["sample_A","sample_B"]
pro_pep = ["Protein/","Peptide/"]
for dirs in species_dirs:
    for sample in samples:
        for p in pro_pep:
            path_dir = os.path.join(path,dirs,sample,p)
            files= os.listdir(path_dir)
            rsd_path = os.path.join(path_dir,"RSD/")
            print(rsd_path)
            rsd_data = pd.DataFrame()
            for i in files:
                if os.path.splitext(i)[1] == '.csv':
                    file = path_dir+i
                    lab = i.split("peptides_after_MVI_")[1].split(".csv")[0]
                    MSFile = pd.read_csv(file,low_memory=False)
                    group_merge = group.merge(pd.DataFrame(MSFile.columns,columns=[0]))
                    MSFile = MSFile.loc[:,group_merge[0]]
                    rsd_data[lab] = MSFile.apply(lambda x: np.std(x,ddof=1) / np.mean(x),axis=1)
            pd.melt(rsd_data.dropna()).to_csv(os.path.join(rsd_path, "RSD_LAB.csv"),index=None)
            print("finish")
            os.system('Rscript /home/zhl/projects/batch_effect_correction/new_results/R、RSD_mod.R {0} {1}'.format(os.path.join(rsd_path, "RSD_LAB.csv"),
                                                                                                os.path.join(rsd_path, "RSD_LAB.png")))
'''

##  7. Ratio

### **All methods of batch-correction in boxplot**

In [45]:
protein_dir = "/home/zhl/projects/batch_effect_correction/new_results/protein_after_batch"
file_dirs = os.listdir(protein_dir)
files = [i for i in file_dirs if os.path.splitext(i)[1] == '.csv']

path = "/home/zhl/projects/batch_effect_correction/new_results/Ratio_3species_allmethod/"
species_dirs = ["swissprot_homo_201604","Saccharomyces cerevisiae_201602","uniprot-Ecoli_K12_201709"]


for species in species_dirs:

    Ratio_data = pd.DataFrame()
    for i in files:
        df = pd.read_csv(os.path.join(protein_dir,i))
        #df = df.loc[:,['Protein','PG.FASTAName']+list(df.columns[df.columns.str.contains("ISAS")])]
        A = df.columns[~df.columns.str.contains("_B_")]
        B = df.columns[~df.columns.str.contains("_A_")]
        
        df_A = df.loc[df["PG.FASTAName"]==species,:]
        df_A = df_A[A]
        
        df_B = df.loc[df["PG.FASTAName"]==species,:]
        df_B = df_B[B]
        
        method = i.split("protein_after_batch_")[1].split(".csv")[0]
        df_AB = pd.merge(df_A,df_B,on = df_A.columns[0])
        Ratio_data[method] = df_AB[df_AB.columns[df_AB.columns.str.contains("_A_")]].T.median() / df_AB[df_AB.columns[df_AB.columns.str.contains("_B_")]].T.median()
    if species == "Saccharomyces cerevisiae_201602":
        species = "Saccharomyces_cerevisiae_201602"
    save_file = os.path.join(path,species+"_Ratio.csv")
    save_png = os.path.join(path,species+"_Ratio.png")
    save_pdf = os.path.join(path,species+"_Ratio.pdf")
    Ratio_data.to_csv(save_file)
    os.system('Rscript /home/zhl/projects/batch_effect_correction/new_results/R/7.Ratio_boxplot.R {0} {1} {2}'.format(save_file,save_png,save_pdf))
    print(save_file," finish!!")

No id variables; using all as measure variables
Warning message:
Removed 41 rows containing non-finite values (stat_boxplot). 
Warning message:
Removed 41 rows containing non-finite values (stat_boxplot). 


/home/zhl/projects/batch_effect_correction/new_results/Ratio_3species_allmethod/swissprot_homo_201604_Ratio.csv  finish!!


No id variables; using all as measure variables
Warning message:
Removed 51 rows containing non-finite values (stat_boxplot). 
Warning message:
Removed 51 rows containing non-finite values (stat_boxplot). 


/home/zhl/projects/batch_effect_correction/new_results/Ratio_3species_allmethod/Saccharomyces_cerevisiae_201602_Ratio.csv  finish!!


No id variables; using all as measure variables


/home/zhl/projects/batch_effect_correction/new_results/Ratio_3species_allmethod/uniprot-Ecoli_K12_201709_Ratio.csv  finish!!


Warning message:
Removed 18 rows containing non-finite values (stat_boxplot). 
Warning message:
Removed 18 rows containing non-finite values (stat_boxplot). 


### **Each method in the scatter plot contains all species**

In [53]:
save_path = "/home/zhl/projects/batch_effect_correction/new_results/Ratio_allspecies_7methods/"
protein_dir = "/home/zhl/projects/batch_effect_correction/new_results/protein_after_batch/"
file_dirs = os.listdir(protein_dir)
files = [i for i in file_dirs if os.path.splitext(i)[1] == '.csv']
species_dirs = ["swissprot_homo_201604","Saccharomyces cerevisiae_201602","uniprot-Ecoli_K12_201709"]

for i in files:
    print(i)
    method = i.split("protein_after_batch_")[1].split(".csv")[0]
    print(method)
    df = pd.read_csv(os.path.join(protein_dir,i),index_col=0)
    Ratio_data = pd.DataFrame(index=df.index)
    
    save_file = os.path.join(save_path,"Protein_Ratio_"+method + ".csv")
    save_png = os.path.join(save_path,method + "_Ratio.png")
    save_pdf = os.path.join(save_path,method + "_Ratio.pdf")
    for species in species_dirs:
        print(species)
        A = df.columns[df.columns.str.contains("_A_")]
        B = df.columns[df.columns.str.contains("_B_")]
        df_A = df.loc[df["PG.FASTAName"]==species,:]
        df_A = df_A[A]
        df_B = df.loc[df["PG.FASTAName"]==species,:]
        df_B = df_B[B]
        df_AB = pd.merge(df_A,df_B,left_index=True,right_index=True)
        
        df_median = np.log2(df_AB.T.median())
        
        df_ratio = df_AB[df_AB.columns[df_AB.columns.str.contains("_A_")]].T.median() / df_AB[df_AB.columns[df_AB.columns.str.contains("_B_")]].T.median()
        df_ratio= np.log2(df_ratio)
        
        Ratio_data[species+"_log2median"] = df_median
        Ratio_data[species] = df_ratio
    Ratio_data.to_csv(save_file)
    os.system('Rscript /home/zhl/projects/batch_effect_correction/new_results/R/8.Ratio_density.R {0} {1} {2}'.format(save_file,save_pdf,save_png))         

protein_after_batch_RUVIII.csv
RUVIII
swissprot_homo_201604
Saccharomyces cerevisiae_201602
uniprot-Ecoli_K12_201709



载入程辑包：‘cowplot’

The following object is masked from ‘package:ggpubr’:

    get_legend



protein_after_batch_RUVg.csv
RUVg
swissprot_homo_201604
Saccharomyces cerevisiae_201602
uniprot-Ecoli_K12_201709



载入程辑包：‘cowplot’

The following object is masked from ‘package:ggpubr’:

    get_legend



protein_after_batch_Combat.csv
Combat
swissprot_homo_201604
Saccharomyces cerevisiae_201602
uniprot-Ecoli_K12_201709


...


载入程辑包：‘cowplot’

The following object is masked from ‘package:ggpubr’:

    get_legend



protein_after_batch_Limma.csv
Limma
swissprot_homo_201604
Saccharomyces cerevisiae_201602
uniprot-Ecoli_K12_201709



载入程辑包：‘cowplot’

The following object is masked from ‘package:ggpubr’:

    get_legend



protein_after_batch_Median-Norm.csv
Median-Norm
swissprot_homo_201604
Saccharomyces cerevisiae_201602
uniprot-Ecoli_K12_201709



载入程辑包：‘cowplot’

The following object is masked from ‘package:ggpubr’:

    get_legend



### single_lab ratio

In [43]:
path = "/home/zhl/projects/batch_effect_correction/new_results/protein_after_batch"
lab_pro = pd.read_csv(os.path.join(path,"protein_after_batch_raw.csv"))
#lab_pro = pd.read_csv(os.path.join(path,"Protein_filter_MVI0.4.csv"),na_values='Filtered')
cols = [i for i in list(lab_pro.columns) if "Day" in i]

In [44]:
labs = []
for col in cols:
    lab = col.split("_",1)[0]
    labs.append(lab)
labs = list(set(labs))

In [56]:
labs

['ISAS',
 'SIMM',
 'WHIRC',
 'ASround1',
 'UniSyd',
 'BGI',
 'MRL',
 'VUMC',
 'ASround2',
 'KISL',
 'FMMU',
 'ETH']

In [72]:
save_path = "/home/zhl/projects/batch_effect_correction/new_results/Ratio_singlelab/"
protein_dir = "/home/zhl/projects/batch_effect_correction/new_results/protein_after_batch/"
file_dirs = os.listdir(protein_dir)
files = [i for i in file_dirs if os.path.splitext(i)[1] == '.csv']
group_labs = group.drop_duplicates(subset=1)
group_labs[0] = group[0].str.replace("_.*","")
species_dirs = ["swissprot_homo_201604","Saccharomyces cerevisiae_201602","uniprot-Ecoli_K12_201709"]
for i in files:
    print(i)
    method = i.split("protein_after_batch_")[1].split(".csv")[0]
    df = pd.read_csv(os.path.join(protein_dir,i),index_col=0)
    for lab in labs:
        Ratio_data = pd.DataFrame(index= df.index)
        print(lab)
        print(method)
        for species in species_dirs:
            print(species)
            A = df.columns[(df.columns.str.contains("_A_"))&df.columns.str.contains(lab)]
            B = df.columns[(df.columns.str.contains("_B_"))&df.columns.str.contains(lab)]
            df_A = df.loc[df["PG.FASTAName"]==species,:]
            df_A = df_A[A]
            df_B = df.loc[df["PG.FASTAName"]==species,:]
            df_B = df_B[B]
            
            df_AB = pd.merge(df_A,df_B,left_index=True,right_index=True)
            df_median = np.log2(df_AB.T.median())
            
            df_ratio = df_AB[df_AB.columns[df_AB.columns.str.contains("_A_")]].T.median() / df_AB[df_AB.columns[df_AB.columns.str.contains("_B_")]].T.median()
            df_ratio = np.log2(df_ratio)
            Ratio_data[species+"_median"] = df_median
            Ratio_data[species] = df_ratio
        lab_ = group_labs[group_labs[0]==lab]
        lab = lab_.iloc[0,1]
        Ratio_data.to_csv(os.path.join(save_path,"Protein_Ratio_"+lab+"_"+method+".csv"))
        os.system('Rscript /home/zhl/projects/batch_effect_correction/new_results/R/8.Ratio_density.R {0} {1} {2}'.format(os.path.join(save_path,"Protein_Ratio_"+lab+"_"+method+".csv"),
                                                                                               os.path.join(save_path,lab+"_"+method+".pdf"),
                                                                                               os.path.join(save_path,lab+"_"+method+".png")))
          

/tmp/ipykernel_50517/391272915.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  group_labs[0] = group[0].str.replace("_.*","")
/tmp/ipykernel_50517/391272915.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  group_labs[0] = group[0].str.replace("_.*","")


protein_after_batch_RUVIII.csv
ISAS
RUVIII
swissprot_homo_201604
Saccharomyces cerevisiae_201602
uniprot-Ecoli_K12_201709



载入程辑包：‘cowplot’

The following object is masked from ‘package:ggpubr’:

    get_legend



SIMM
RUVIII
swissprot_homo_201604
Saccharomyces cerevisiae_201602
uniprot-Ecoli_K12_201709



载入程辑包：‘cowplot’

The following object is masked from ‘package:ggpubr’:

    get_legend



...


载入程辑包：‘cowplot’

The following object is masked from ‘package:ggpubr’:

    get_legend



FMMU
Median-Norm
swissprot_homo_201604
Saccharomyces cerevisiae_201602
uniprot-Ecoli_K12_201709



载入程辑包：‘cowplot’

The following object is masked from ‘package:ggpubr’:

    get_legend



ETH
Median-Norm
swissprot_homo_201604
Saccharomyces cerevisiae_201602
uniprot-Ecoli_K12_201709



载入程辑包：‘cowplot’

The following object is masked from ‘package:ggpubr’:

    get_legend



# <font color=green>TOP3 RATIO

No impute

In [ ]:
'''
缺失值填充
lab_pep = pd.read_table("/home/boot/Script/test/batch_pipeline/batch_effect_correction/NC/Ratio_method/single_lab/proteins_1-Sample_Report.xls",
                       na_values='Filtered')
lab_pep.index = lab_pep["PG.ProteinGroups"]
lab_pep_num = lab_pep.iloc[:,5:]
lab_pep_num[lab_pep_num <= 100] = np.nan
lab_pep_num.dropna(axis=0,thresh=lab_pep_num.shape[1]*0.5,inplace = True)
#lab_pep_num.dropna(axis=0,how="any",inplace = True)
lab_pep_num = lab_pep_num.T
lab_pep_impute = fancyimpute.KNN(k=3).fit_transform(lab_pep_num.values) #knn按列进行填充
lab_pep_impute = pd.DataFrame(lab_pep_impute, columns=list(lab_pep_num.columns), index=lab_pep_num.index).T
lab_pep = pd.merge(lab_pep.loc[:,["PG.ProteinGroups","PG.Organisms"]],lab_pep_impute,
                         how="right",left_index = True,right_index = True)
lab_pep.reset_index(drop=True,inplace=True)'''



# Precision

In [3]:
MSFile = pd.read_csv('/home/zhl/projects/batch_effect_correction/new_results/protein_after_batch/protein_after_batch_RUVIII.csv',low_memory=False)
group=pd.read_excel("/home/zhl/projects/batch_effect_correction/new_results/group_new.xls",header=None)
group = group.iloc[1:,0:2]
group = group.merge(pd.DataFrame(MSFile.columns,columns=[0]))
group

,0,1
0,FMMU_Day_1_A_rep_1,Lab1
1,FMMU_Day_1_A_rep_2,Lab1
2,FMMU_Day_1_A_rep_3,Lab1
3,FMMU_Day_1_B_rep_1,Lab1
4,FMMU_Day_1_B_rep_2,Lab1
...,...,...
300,MRL_Day_7_A_Rep_2,Lab11
301,MRL_Day_7_A_Rep_3,Lab11
302,MRL_Day_7_B_Rep_1,Lab11
303,MRL_Day_7_B_Rep_2,Lab11


In [7]:
path = "/home/zhl/projects/batch_effect_correction/new_results/protein_after_batch"
lab_pro = pd.read_csv(os.path.join(path,"protein_after_batch_raw.csv"))
#lab_pro = pd.read_csv(os.path.join(path,"Protein_filter_MVI0.4.csv"),na_values='Filtered')
cols = [i for i in list(lab_pro.columns) if "Day" in i]

In [8]:
labs = []
for col in cols:
    lab = col.split("_",1)[0]
    labs.append(lab)
labs = list(set(labs))

In [ ]:
save_path = "/home/zhl/projects/batch_effect_correction/new_results/precision/"
protein_dir = "/home/zhl/projects/batch_effect_correction/new_results/protein_after_batch/"
file_dirs = os.listdir(protein_dir)
files = [i for i in file_dirs if os.path.splitext(i)[1] == '.csv']
group_labs = group.drop_duplicates(subset=1)
group_labs[0] = group[0].str.replace("_.*","")
species_dirs = ["swissprot_homo_201604","Saccharomyces cerevisiae_201602","uniprot-Ecoli_K12_201709"]

In [99]:
df_rsd_median = pd.DataFrame(columns=["method","species","median rsd"])
for i in files:
    print(i)
    method = i.split("protein_after_batch_")[1].split(".csv")[0]
    df = pd.read_csv(os.path.join(protein_dir,i),index_col=0)
 
    subgroup = group[group[0].str.contains("_A_")]
    rsd_data = pd.DataFrame()
    for species in species_dirs:
        print(species)
        A_sample = df.columns[(df.columns.str.contains("_A_"))]
        df_A = df.loc[df["PG.FASTAName"]==species,:]
        df_A = df_A[A_sample]
        group2sample = subgroup.groupby(1,as_index=False)
        
        for key, value in group2sample:
            Value = list(value[0])
            sub_rsd_data = df_A[Value].apply(lambda x: np.std(x[Value], ddof=1) / np.mean(x[Value]),axis=1)
            sub_rsd_data = pd.DataFrame(sub_rsd_data)
        sub_rsd_data["species"] = species
        sub_rsd_data.columns = ["RSD","species"]
        rsd_data = pd.concat([rsd_data,sub_rsd_data])
        
        median_rsd = np.median(np.median(sub_rsd_data["RSD"]))
        s_df_rsd_median = pd.DataFrame({"method":method,"species":species,"median rsd":median_rsd},index=[0])
        df_rsd_median = pd.concat([df_rsd_median,s_df_rsd_median])

    #break
    rsd_data.to_csv(os.path.join(save_path, 'rsd_A_'+i))
df_rsd_median.to_csv(os.path.join(save_path, 'rsd_median_A.csv'),index=None)
    #np.median(rsd_data[rsd_data["species"]=="swissprot_homo_201604"].loc[:,"RSD"])

protein_after_batch_RUVIII.csv
swissprot_homo_201604
Saccharomyces cerevisiae_201602
uniprot-Ecoli_K12_201709
protein_after_batch_RUVg.csv
swissprot_homo_201604
Saccharomyces cerevisiae_201602
uniprot-Ecoli_K12_201709
protein_after_batch_Combat.csv
swissprot_homo_201604
Saccharomyces cerevisiae_201602
uniprot-Ecoli_K12_201709
protein_after_batch_QC-Mnorm.csv
swissprot_homo_201604
Saccharomyces cerevisiae_201602
uniprot-Ecoli_K12_201709
protein_after_batch_raw.csv
swissprot_homo_201604
Saccharomyces cerevisiae_201602
uniprot-Ecoli_K12_201709
protein_after_batch_RUVs.csv
swissprot_homo_201604
Saccharomyces cerevisiae_201602
uniprot-Ecoli_K12_201709
protein_after_batch_Median-Norm.csv
swissprot_homo_201604
Saccharomyces cerevisiae_201602
uniprot-Ecoli_K12_201709


In [100]:
df_rsd_median = pd.DataFrame(columns=["method","species","median rsd"])
for i in files:
    print(i)
    method = i.split("protein_after_batch_")[1].split(".csv")[0]
    df = pd.read_csv(os.path.join(protein_dir,i),index_col=0)
 
    subgroup = group[group[0].str.contains("_B_")]
    rsd_data = pd.DataFrame()
    for species in species_dirs:
        print(species)
        A_sample = df.columns[(df.columns.str.contains("_B_"))]
        df_A = df.loc[df["PG.FASTAName"]==species,:]
        df_A = df_A[A_sample]
        group2sample = subgroup.groupby(1,as_index=False)
        
        for key, value in group2sample:
            Value = list(value[0])
            sub_rsd_data = df_A[Value].apply(lambda x: np.std(x[Value], ddof=1) / np.mean(x[Value]),axis=1)
            sub_rsd_data = pd.DataFrame(sub_rsd_data)
        sub_rsd_data["species"] = species
        sub_rsd_data.columns = ["RSD","species"]
        rsd_data = pd.concat([rsd_data,sub_rsd_data])
        
        median_rsd = np.median(np.median(sub_rsd_data["RSD"]))
        s_df_rsd_median = pd.DataFrame({"method":method,"species":species,"median rsd":median_rsd},index=[0])
        df_rsd_median = pd.concat([df_rsd_median,s_df_rsd_median])

    #break
    rsd_data.to_csv(os.path.join(save_path, 'rsd_B_'+i))
df_rsd_median.to_csv(os.path.join(save_path, 'rsd_median_B.csv'),index=None)


protein_after_batch_RUVIII.csv
swissprot_homo_201604
Saccharomyces cerevisiae_201602
uniprot-Ecoli_K12_201709
protein_after_batch_RUVg.csv
swissprot_homo_201604
Saccharomyces cerevisiae_201602
uniprot-Ecoli_K12_201709
protein_after_batch_Combat.csv
swissprot_homo_201604
Saccharomyces cerevisiae_201602
uniprot-Ecoli_K12_201709
protein_after_batch_QC-Mnorm.csv
swissprot_homo_201604
Saccharomyces cerevisiae_201602
uniprot-Ecoli_K12_201709
protein_after_batch_raw.csv
swissprot_homo_201604
Saccharomyces cerevisiae_201602
uniprot-Ecoli_K12_201709
protein_after_batch_RUVs.csv
swissprot_homo_201604
Saccharomyces cerevisiae_201602
uniprot-Ecoli_K12_201709
protein_after_batch_Median-Norm.csv
swissprot_homo_201604
Saccharomyces cerevisiae_201602
uniprot-Ecoli_K12_201709


In [97]:
df_rsd_median.to_csv(os.path.join(save_path, 'rsd_median_A.csv'),index=None)

In [116]:
file_ab[file_ab["species"]=="swissprot_homo_201604"]

,Protein,RSD,species
0,A0A0B4J2F0,0.095190,swissprot_homo_201604
1,A0AVT1,0.043000,swissprot_homo_201604
2,A0FGR8,0.098165,swissprot_homo_201604
3,A0JLT2,0.144951,swissprot_homo_201604
4,A0JNW5,0.493554,swissprot_homo_201604
...,...,...,...
5021,Q9Y6W3,0.204156,swissprot_homo_201604
5022,Q9Y6W5,0.202600,swissprot_homo_201604
5023,Q9Y6X3,0.260800,swissprot_homo_201604
5024,Q9Y6X9,0.125548,swissprot_homo_201604


In [127]:
save_path = "/home/zhl/projects/batch_effect_correction/new_results/precision/ab"
dirs = "/home/zhl/projects/batch_effect_correction/new_results/precision/"
file_dirs = os.listdir(dirs)
files = [i for i in file_dirs if "protein_after_batch_" in i]
methods = ["raw","Combat","Median-Norm","QC-Mnorm","RUVs","RUVg","RUVIII"]
for m in methods:
    print(m)
    file_method = [i for i in files if i.split("protein_after_batch_")[1].split(".csv")[0]==m]
    file_a = pd.read_csv(os.path.join(dirs,file_method[0]))
    file_a["sample"] = "A"
    file_b = pd.read_csv(os.path.join(dirs,file_method[1]))
    file_b["sample"] = "B"
    file_ab = pd.concat([file_a,file_b])
    file_ab.to_csv(os.path.join(save_path,m + "_rsd.csv"),index=None)
    rsd_human = file_ab[file_ab["species"]=="swissprot_homo_201604"].loc[:,"RSD"]
    rsd_yeast = file_ab[file_ab["species"]=="Saccharomyces cerevisiae_201602"].loc[:,"RSD"]
    rsd_ecoli = file_ab[file_ab["species"]=="uniprot-Ecoli_K12_201709"].loc[:,"RSD"]
    
    rsd_median_human = np.median(rsd_human)
    rsd_median_yeast = np.median(rsd_yeast)
    rsd_median_ecoli = np.median(rsd_ecoli)
    
    print("human:",round(rsd_median_human,3))
    print("yeast:",round(rsd_median_yeast,3))
    print("ecoli:",round(rsd_median_ecoli,3),"\n")

raw
human: 0.1
yeast: 0.124
ecoli: 0.131 

Combat
human: 0.123
yeast: 0.154
ecoli: 0.18 

Median-Norm
human: 0.133
yeast: 0.155
ecoli: 0.172 

QC-Mnorm
human: 0.151
yeast: 0.174
ecoli: 0.179 

RUVs
human: 0.155
yeast: 0.244
ecoli: 0.245 

RUVg
human: 0.111
yeast: 0.14
ecoli: 0.175 

RUVIII
human: 0.126
yeast: 0.197
ecoli: 0.316 



In [140]:
import scipy.stats as stats
os.chdir(save_path)
file_dirs = os.listdir(save_path)
files = [i for i in file_dirs if i.split("_rsd")[0]=="Median-Norm" or i.split("_rsd")[0]=="Combat"]
#methods = ["raw","Combat","Median-Norm","QC-Mnorm","RUVs","RUVg","RUVIII"]
file_1 = pd.read_csv(files[0])
file_2 = pd.read_csv(files[1])
rsd_human1 = file_1[file_1["species"]=="swissprot_homo_201604"].loc[:,"RSD"]
rsd_human2 = file_2[file_2["species"]=="swissprot_homo_201604"].loc[:,"RSD"]
stats.wilcoxon(rsd_human1,rsd_human2,alternative='two-sided')
stats.ttest_ind(rsd_human1,rsd_human2)
#rsd_yeast = file_ab[file_ab["species"]=="Saccharomyces cerevisiae_201602"].loc[:,"RSD"]
#rsd_ecoli = file_ab[file_ab["species"]=="uniprot-Ecoli_K12_201709"].loc[:,"RSD"]

Ttest_indResult(statistic=5.144416758983781, pvalue=2.708744079802156e-07)